In [1]:
import numpy as np
import scrapy

In [2]:
from scrapy.crawler import CrawlerProcess

In [3]:
a = np.loadtxt('link_hoteles.txt', dtype='str')

In [4]:
class Title(scrapy.Spider):
    name = 'riviera_data'
    final_links = np.loadtxt('link_hoteles.txt', dtype='str')
    start_urls = final_links.tolist()
    custom_settings = {
        'FEED_URI': 'riviera_data.json',
        'FEED_FORMAT': 'json',
        'FEED_EXPORT_ENCODING': 'utf-8',
        'ROBOTSTXT_OBEY': True,
    }

    LINK_TITLE = '//div[@class = "et_pb_title_container"]/h1[@class = "entry-title"]/text()'
    LINK_PARAGRAPHS = '//div[@class = "et_pb_module et_pb_post_content et_pb_post_content_0_tb_body"]/p/text()'
    LINK_DIRECTION = '//div[@class = "et_pb_module et_pb_post_content et_pb_post_content_0_tb_body"]/p/text()[preceding-sibling::strong]'
    LINK_MAIL = '//div[@class = "et_pb_module et_pb_post_content et_pb_post_content_0_tb_body"]/p/a/@href'
    LINK_TEL = '//div[@class = "et_pb_module et_pb_post_content et_pb_post_content_0_tb_body"]/p/strong[contains(.,"Tel")]/text()'
    LINK_FAX = '//div[@class = "et_pb_module et_pb_post_content et_pb_post_content_0_tb_body"]/p/strong[contains(.,"Fax")]/text()'

    def parse(self, response):
        # get the title
        title = response.xpath(self.LINK_TITLE).get()
        # get all the paragraphs
        paragraphs = response.xpath(self.LINK_PARAGRAPHS).getall()
        # get the first line of the direction
        start_dir = response.xpath(self.LINK_DIRECTION).get()
        # filter only the direction
        val = paragraphs.index(start_dir)
        paragraphs = paragraphs[val:]
        for i in paragraphs:
            if "@" in i or " " in i:
                paragraphs.pop(paragraphs.index(i))
        # concatenate the direction
        paragraphs = " ".join(paragraphs)
        # get the mails
        mail_href = response.xpath(self.LINK_MAIL).getall()
        web_site = mail_href[1]
        mail_href = mail_href[0]
        mail_href = mail_href[7:]
        # mail_all = []
        # for i in mail_href:
        #     for j in i:
        #         if "@" in i:
        #             mail_all.append(i)
        #         else:
        #             mail_all.append("NULL")
        phones =  response.xpath(self.LINK_TEL).get()
        fax = response.xpath(self.LINK_FAX).get()
        #filter data
        phones = phones[5:]
        fax = fax[5:]
        # delete blank spaces
        paragraphs = paragraphs.replace(u'\xa0', u' ')
        paragraphs = paragraphs.replace('%20', '')
        mail_href = mail_href.replace(u'\xa0', u' ')
        mail_href = mail_href.replace('%20', '')
        phones = phones.replace(u'\xa0', u' ')
        fax = fax.replace(u'\xa0', u' ')
        web_site = web_site.replace(u'\xa0', u' ')
        yield {
            'Hotel': title,
            'Dirección': paragraphs,
            'mail' : mail_href,
            'Teléfono' : phones,
            'Fax' : fax,
            'Website': web_site
        }

In [5]:
process = CrawlerProcess({
    'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'
})

process.crawl(Title)
process.start()

2022-10-22 15:12:48 [scrapy.utils.log] INFO: Scrapy 2.7.0 started (bot: scrapybot)
2022-10-22 15:12:48 [scrapy.utils.log] INFO: Versions: lxml 4.9.1.0, libxml2 2.9.12, cssselect 1.1.0, parsel 1.6.0, w3lib 2.0.1, Twisted 22.8.0, Python 3.10.7 (tags/v3.10.7:6cc6b13, Sep  5 2022, 14:08:36) [MSC v.1933 64 bit (AMD64)], pyOpenSSL 22.1.0 (OpenSSL 3.0.5 5 Jul 2022), cryptography 38.0.1, Platform Windows-10-10.0.19043-SP0
2022-10-22 15:12:48 [scrapy.crawler] INFO: Overridden settings:
{'FEED_EXPORT_ENCODING': 'utf-8',
 'ROBOTSTXT_OBEY': True,
 'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'}
2022-10-22 15:12:48 [py.warnings] WARNING: D:\Platzi\Web-scraping-Xpath\venv\lib\site-packages\scrapy\utils\request.py:231: ScrapyDeprecationWarning: '2.6' is a deprecated value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting.

It is also the default value. In other words, it is normal to get this warning if you have not defined a value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATIO